# Zero-Coupon Bond Using Marlowe Runtime\'s REST API

The zero-coupon bond example is a simple Marlowe contract where a lender provides principal to a borrower who repays it back with interest.

In this demonsration we use Marlowe Runtime\'s REST API, served via `marlowe-web-server`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use role tokens and we have Marlowe Runtime mint them.

In [Marlowe Playground](https://play.marlowe-finance.io/), the contract looks like this in Blockly format.

![Zero-coupon bond Marlowe contract](zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Lender")
            (Role "Lender")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Role "Lender")
            (Party (Role "Borrower"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Role "Borrower")
                        (Role "Borrower")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Role "Borrower")
                        (Party (Role "Lender"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close
```

## Prelminaries

See [Lesson 0. Preliminaries](../00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.
- `CARDANO_RT_WEBSERVER_HOST`: IP address of the Marlowe Runtime web server.
- `CARDANO_RT_WEBSERVER_PORT`: Port number for the Marlowe Runtime web server.

It also assumes that the Lender and Borrower parties have addresses, signing keys, and funds.
- Lender
    - [../keys/lender.address](../keys/lender.address): Cardano address for the lender
    - [../keys/lender.skey](../keys/lender.skey): location of signing key file for the lender
- Borrower
    - [../keys/borrower.address](../keys/borrower.address): Cardano address for the borrower
    - [../keys/borrower.skey](../keys/borrower.skey): location of signing key file for the borrower

### Access to Cardano node and Marlowe Runtime

***One can skip this section if one is using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension.***

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. If not, set the required environment variables to use a local docker deployment on the default ports.

In [60]:
if [[ -z "$MARLOWE_RT_WEBSERVER_PORT" ]]
then

  if command -v podman > /dev/null
  then
    DOCKER_CLI=podman
  else
    DOCKER_CLI=docker
  fi
  
  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$($DOCKER_CLI volume inspect marlowe-compose_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for `marlowe`.
  export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
  export MARLOWE_RT_WEBSERVER_PORT=3780

fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST:$MARLOWE_RT_WEBSERVER_PORT"

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_WEBSERVER_HOST = $MARLOWE_RT_WEBSERVER_HOST"
echo "MARLOWE_RT_WEBSERVER_PORT = $MARLOWE_RT_WEBSERVER_PORT"
echo "MARLOWE_RT_WEBSERVER_URL = $MARLOWE_RT_WEBSERVER_URL"

CARDANO_NODE_SOCKET_PATH = /home/bbush/.local/share/containers/storage/volumes/marlowe-compose_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1
MARLOWE_RT_WEBSERVER_HOST = 127.0.0.1
MARLOWE_RT_WEBSERVER_PORT = 3780
MARLOWE_RT_WEBSERVER_URL = http://127.0.0.1:3780


### Lender address and funds

Check that an address and key has been created for the lender. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](../00-preliminaries.md).

In [61]:
LENDER_SKEY=../keys/lender.skey
LENDER_ADDR=$(cat ../keys/lender.address)
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


Check that the lender has at least one hundred ada.

In [62]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dc380bc79a628167244f785e32a7750b007dd0084970a19fc58e22ca2ce190b5     2        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [63]:
echo "$EXPLORER_URL/address/$LENDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Borrower address and funds

Check that an address and key has been created for the borrower. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](../00-preliminaries.md).

In [64]:
BORROWER_SKEY=../keys/borrower.skey
BORROWER_ADDR=$(cat ../keys/borrower.address)
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


Check that the borrower has at least one hundred ada.

In [65]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dc380bc79a628167244f785e32a7750b007dd0084970a19fc58e22ca2ce190b5     3        1000000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [66]:
echo "$EXPLORER_URL/address/$BORROWER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


## Design the contract

The zero-coupon bond contract can be downloaded from the [Marlowe Playground](https://play.marlowe-finance.io/) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Set the loan's principal to 80 ada and it's interest to 5 ada.

In [67]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))
echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

PRINCIPAL = 80000000 lovelace
INTEREST = 5000000 lovelace


On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [68]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Later in the example we will need some constants for converting times.

In [69]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

### *Alternative 1:* Use Marlowe Playground to design the contract

If you want to create the contract in Marlowe Playground, do the following:

1. Visit https://play.marlowe-finance.io/ in a web browser.
2. Select "Open an Example".
3. Select "Marlowe" or "Blockly" under "Zero Coupon Bond".
4. Select "Send to Simulator".
5. Set the "Loan Deadline" to one hour into the future.
6. Set the "Payback Deadline" to three hours into the future.
7. Set the "Principal" to 80 ada.
8. Set the "Interest" to 5 ada.
9. Select "Download as JSON", set the file name to "zcb-contract.json", and store the file in this folder, namely [marlowe-starter-kit/01-runtime-rest/](.).

![Setting parameters for the zero-coupon bond contract in Marlowe Playground](../images/zcb-playground.png)

### *Alternative 2:* Use Marlowe CLI to generate the contract

Below we generate the contract using Marlowe CLI.

First find the current time, measured in [POSIX milliseconds](https://en.wikipedia.org/wiki/Unix_time).

In [70]:
NOW="$(($(date -u +%s) * SECOND))"
echo "NOW = $NOW POSIX milliseconds = $(date -d @$((NOW / SECOND)))"

NOW = 1678903540000 POSIX milliseconds = Wed Mar 15 12:05:40 PM MDT 2023


The contract has a lending deadline and a repayment deadline. For convenience in this example, set the deadlines to the near future.

In [71]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo "LENDER_DEADLINE = $LENDER_DEADLINE POSIX milliseconds = $(date -d @$((LENDER_DEADLINE / SECOND)))"
echo "BORROWER_DEADLINE = $BORROWER_DEADLINE POSIX milliseconds = $(date -d @$((BORROWER_DEADLINE / SECOND)))"

LENDER_DEADLINE = 1678907140000 POSIX milliseconds = Wed Mar 15 01:05:40 PM MDT 2023
BORROWER_DEADLINE = 1678914340000 POSIX milliseconds = Wed Mar 15 03:05:40 PM MDT 2023


Now create the JSON file for the contract, `zcb-contract.json`.

In [72]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender Lender \
  --borrower Borrower \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file /dev/null

The various command-line options are described by the help system.

In [73]:
marlowe-cli template zcb --help

Usage: marlowe-cli template zcb --minimum-ada INTEGER --lender PARTY
                                --borrower PARTY --principal INTEGER
                                --interest INTEGER --lending-deadline TIMEOUT
                                --repayment-deadline TIMEOUT

  Create a zero-coupon bond.

Available options:
  --minimum-ada INTEGER    Lovelace that the lender contributes to the initial
                           state.
  --lender PARTY           The lender.
  --borrower PARTY         The borrower.
  --principal INTEGER      The principal, in lovelace.
  --interest INTEGER       The interest, in lovelace.
  --lending-deadline TIMEOUT
                           The lending deadline. POSIX milliseconds or duration:
                           `INTEGER[s|m|d|w|h]`.
  --repayment-deadline TIMEOUT
                           The repayment deadline. POSIX milliseconds or
                           duration: `INTEGER[s|m|d|w|h]`.
  -h,--help                Show this help text


## Examine the contract

View the contract file as YAML.

In [74]:
json2yaml zcb-contract.json

timeout: 1678907140000
timeout_continuation: close
when:
- case:
    deposits: 80000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Lender
  then:
    from_account:
      role_token: Lender
    pay: 80000000
    then:
      timeout: 1678914340000
      timeout_continuation: close
      when:
      - case:
          deposits:
            add: 80000000
            and: 5000000
          into_account:
            role_token: Borrower
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            role_token: Borrower
        then:
          from_account:
            role_token: Borrower
          pay:
            add: 80000000
            and: 5000000
          then: close
          to:
            party:
              role_token: Lender
          token:
            currency_symbol: ''
            token_name: ''
    to:
      party:
        role_token: Borro

### \[Optional\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe-finance.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe-finance.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](../01-runtime-cli/ReadMe.ipynb) for an example of performing step 6.

## Transaction 1. Create the Contract

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

First we create the JSON body of the request to build the creation transaction.

In [75]:
yaml2json << EOI > request-1.json
version: v1
contract: $(cat zcb-contract.json)
roles:
  Lender: $LENDER_ADDR
  Borrower: $BORROWER_ADDR
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json

{"contract":{"timeout":1678907140000,"timeout_continuation":"close","when":[{"case":{"deposits":80000000,"into_account":{"role_token":"Lender"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Lender"}},"then":{"from_account":{"role_token":"Lender"},"pay":80000000,"then":{"timeout":1678914340000,"timeout_continuation":"close","when":[{"case":{"deposits":{"add":80000000,"and":5000000},"into_account":{"role_token":"Borrower"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Borrower"}},"then":{"from_account":{"role_token":"Borrower"},"pay":{"add":80000000,"and":5000000},"then":"close","to":{"party":{"role_token":"Lender"}},"token":{"currency_symbol":"","token_name":""}}}]},"to":{"party":{"role_token":"Borrower"}},"token":{"currency_symbol":"","token_name":""}}}]},"metadata":{},"minUTxODeposit":2000000,"roles":{"Borrower":"addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d","Lender":"addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2

Next we post the request and view the response.

In [76]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa%231
resource:
  contractId: 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1
  txBody:
    cborHex: 86a80081825820dc380bc79a628167244f785e32a7750b007dd0084970a19fc58e22ca2ce190b5020d81825820dc380bc79a628167244f785e32a7750b007dd0084970a19fc58e22ca2ce190b5020184a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b55c4daa300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e848002820058205e40641b9b037b9f920f3346f15db6f9ba0943cdf733cc86ac6ce4d2af968e37a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a000fea4ca1581c68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597a148426f72726f77657201a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a000fc8a0a1581c68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597a1464c656e6465720110a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b

The identifier for the contract is embedded in the response.

In [77]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1


The CBOR serialization (in text-envelope format) is also embedded in the response.

In [78]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [79]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa


One can view the transaction on a Cardano explorer and see that the contract has been created and the parties have received their role tokens. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [80]:
echo "$EXPLORER_URL/transaction/$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa?tab=utxo


In particular, we see that the Marlowe contract holds the 2 ada that was set as `MINIMUM_LOVELACE`.

In [81]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "5e40641b9b037b9f920f3346f15db6f9ba0943cdf733cc86ac6ce4d2af968e37"


One can see that the lender and borrower have received their role tokens. Note that `4c656e646572 = Lender` and `426f72726f776572 = Borrower` in hexadecimal notation.

In [82]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa     0        995476698 lovelace + TxOutDatumNone
97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa     3        1034400 lovelace + 1 68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597.4c656e646572 + TxOutDatumNone


In [83]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa     2        1043020 lovelace + 1 68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597.426f72726f776572 + TxOutDatumNone
dc380bc79a628167244f785e32a7750b007dd0084970a19fc58e22ca2ce190b5     3        1000000000 lovelace + TxOutDatumNone


## View the details of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}` can fetch a contract from the blockchain and return information about it.

In [84]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/$(jq -r '.links.contract' response-1.json)"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://127.0.0.1:3780/contracts/97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa%231


In [85]:
curl -sS "$CONTRACT_URL" | json2yaml

links:
  transactions: contracts/97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa%231/transactions
resource:
  block:
    blockHeaderHash: 89ac1a96f734eab3ed8723292576be23867da464f85d0789d8453a662c2cacfe
    blockNo: 728272
    slotNo: 23220386
  continuations: null
  contractId: 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1
  currentContract:
    timeout: 1678907140000
    timeout_continuation: close
    when:
    - case:
        deposits: 80000000
        into_account:
          role_token: Lender
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Lender
      then:
        from_account:
          role_token: Lender
        pay: 80000000
        then:
          timeout: 1678914340000
          timeout_continuation: close
          when:
          - case:
              deposits:
                add: 80000000
                and: 5000000
              into_account:
                role_token

## Transaction 2. The lender deposits the principal

The lender deposits their 80 ada of principal into the contract using Marlowe Runtime\'s `HTTP` `POST` `/contract/{contractId}/transactions` endpoint. The lender is providing the funding for and receiving the change from this transaction, so we provide their address.

The deposit is represented as JSON input to the contract. The `marlowe-cli input deposit` tool conveniently formats the correct JSON for a deposit.

In [86]:
marlowe-cli input deposit --help

Usage: marlowe-cli input deposit --deposit-account PARTY --deposit-party PARTY 
                                 [--deposit-token TOKEN]
                                 --deposit-amount INTEGER 
                                 [--out-file OUTPUT_FILE]

  Create Marlowe input for a deposit.

Available options:
  --deposit-account PARTY  The account for the deposit.
  --deposit-party PARTY    The party making the deposit.
  --deposit-token TOKEN    The token being deposited, if not Ada.
  --deposit-amount INTEGER The amount of token being deposited.
  --out-file OUTPUT_FILE   JSON output file for contract input.
  -h,--help                Show this help text


In [87]:
marlowe-cli input deposit \
  --deposit-party Lender \
  --deposit-account Lender \
  --deposit-amount "$PRINCIPAL" \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Lender
into_account:
  role_token: Lender
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 80000000


This input is included in the JSON request.

In [88]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Lender"},"into_account":{"role_token":"Lender"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":80000000}],"metadata":{},"tags":{},"version":"v1"}


Next we post the request and store the response.

In [89]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

links:
  transaction: contracts/97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa%231/transactions/dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc
resource:
  contractId: 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1
  transactionId: dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc
  txBody:
    cborHex: 86aa008382582097739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa0082582097739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa0182582097739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa030d8182582097739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010184a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a36844ba7a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e848002820058203c74acfcff4466c5b7cd05427375b900ccfa74a4d058d3fffe1f1cbdfc95aa5fa200581d601b120

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [90]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [91]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [92]:
echo "$EXPLORER_URL/transaction/$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc?tab=utxo


One can see that the lender has approximately 83 ada less than originally. Two ada were deposited in the contract when it was created and 80 ada were paid to the borrower in the second transaction; another 1 ada was attached to the role token that was sent to the borrower. The lender also holds their own role token.

In [93]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc     0        914639783 lovelace + TxOutDatumNone
dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc     2        1034400 lovelace + 1 68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597.4c656e646572 + TxOutDatumNone


The Marlowe contract still has the 2 ada from its creation.

In [94]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "3c74acfcff4466c5b7cd05427375b900ccfa74a4d058d3fffe1f1cbdfc95aa5f"


Marlowe\'s role-payout address holds the 80 ada on behalf of the borrower.

In [95]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#3"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc     3        80000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "7ebb64002f2b303771515baeecce5eb15069b90fbe6393fd3a8cba1c5607214f"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [96]:
curl -sS "$CONTRACT_URL/transactions/$TX_2" | json2yaml

links: {}
resource:
  block:
    blockHeaderHash: 6ba6ee90728b334ddf70a06ea434899a345bda598fe334194dea801856d873a1
    blockNo: 728274
    slotNo: 23220478
  consumingTx: null
  continuations: null
  contractId: 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1
  inputUtxo: 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1
  inputs:
  - input_from_party:
      role_token: Lender
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 80000000
  invalidBefore: 2023-03-15T18:06:54Z
  invalidHereafter: 2023-03-15T19:05:40Z
  metadata: {}
  outputContract:
    timeout: 1678914340000
    timeout_continuation: close
    when:
    - case:
        deposits:
          add: 80000000
          and: 5000000
        into_account:
          role_token: Borrower
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Borrower
      then:
        fr

## Transaction 3. The borrower withdraws the principal

The principal of 80 ada is held at Marlowe's role-payout address for the benefit of the borrower. The borrower can withdraw these funds at any time. The contract ID and role name are included in the request body for a withdrawal.

In [97]:
yaml2json << EOI > request-3.json
contractId: "$CONTRACT_ID"
role: Borrower
EOI
cat request-3.json

{"contractId":"97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1","role":"Borrower"}


Next we post the request and store the response.

In [98]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BORROWER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

links:
  withdrawal: withdrawals/b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c2416
resource:
  txBody:
    cborHex: 86a8008382582097739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa02825820dc380bc79a628167244f785e32a7750b007dd0084970a19fc58e22ca2ce190b503825820dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc030d81825820dc380bc79a628167244f785e32a7750b007dd0084970a19fc58e22ca2ce190b50312818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a405a8383a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a000fea4ca1581c68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597a148426f72726f7765720110a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b935244111a000777bc021a0004fa7d0b582008647a69a8cf493d8673849f35d61d87a24bb4f8ea871d6fcf5743f6bbcdbcb59fff81d8799f581c68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad5974

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [99]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

In [100]:
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c2416


On can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [101]:
echo "$EXPLORER_URL/transaction/$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c2416?tab=utxo


The borrower now has about an additional 80 ada (the loan's principal). The borrower also holds their own role token.

In [102]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c2416     0        1079673731 lovelace + TxOutDatumNone
b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c2416     1        1043020 lovelace + 1 68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597.426f72726f776572 + TxOutDatumNone


## Transaction 4. The borrower repays the loan

After some time passes, the borrower repays principal plus interest. Thus, they fund the transaction and receive the change at their address.

First build the input to deposit the funds to the contract.

In [103]:
marlowe-cli input deposit \
  --deposit-party Borrower \
  --deposit-account Borrower \
  --deposit-amount "$((PRINCIPAL+INTEREST))" \
  --out-file input-4.json
json2yaml input-4.json

input_from_party:
  role_token: Borrower
into_account:
  role_token: Borrower
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 85000000


Next build the request.

In [104]:
yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat input-4.json)]
metadata: {}
tags: {}
EOI
cat request-4.json

{"inputs":[{"input_from_party":{"role_token":"Borrower"},"into_account":{"role_token":"Borrower"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":85000000}],"metadata":{},"tags":{},"version":"v1"}


Now post the request to Marlowe Runtime.

In [105]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $BORROWER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

links:
  transaction: contracts/97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa%231/transactions/540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd
resource:
  contractId: 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1
  transactionId: 540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd
  txBody:
    cborHex: 86aa0083825820b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c241600825820b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c241601825820dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc010d81825820b450ee66c27ebeaa0ae1928e7163628b27462f33c7c9316c47058185038c24160012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010184a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b3f8180a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a000fea4ca1581c68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597a148426f72726f77657201a30

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [106]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [107]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = 540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [108]:
echo "$EXPLORER_URL/transaction/$TX_4?tab=utxo"

https://preprod.cardanoscan.io/transaction/540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd?tab=utxo


The borrower now has a total of about 5 ada (the loan's interest) less than originally.

In [109]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd     0        994017664 lovelace + TxOutDatumNone
540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd     1        1043020 lovelace + 1 68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597.426f72726f776572 + TxOutDatumNone


The 85 ada are at Marlowe\'s role-payout address, held for the benefit of the lender.

In [110]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_4#2"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd     2        85000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "45f5647eb6ccae98c4f391da5d74b0ccc6bc4aaaca08cf78d3bbe6deb0c77b9d"


The Marlowe contract has closed, so there is no output to its script address.

## View the completion of the contract on the blockchain

Marlowe Runtime\'s `HTTP` `GET` endpoint `/contracts/{contractId}/transactions/{transactionId}` can fetch a contract from the blockchain and return information about it.

In [111]:
curl -sS "$CONTRACT_URL/transactions/$TX_4" | json2yaml

links:
  previous: contracts/97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa%231/transactions/dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc
resource:
  block:
    blockHeaderHash: c393c51801695106acd0bc329a864fec9d8bae8ffbc1c72adfbfb9e55b8a570d
    blockNo: 728277
    slotNo: 23220567
  consumingTx: null
  continuations: null
  contractId: 97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1
  inputUtxo: dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc#1
  inputs:
  - input_from_party:
      role_token: Borrower
    into_account:
      role_token: Borrower
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 85000000
  invalidBefore: 2023-03-15T18:08:59Z
  invalidHereafter: 2023-03-15T21:05:40Z
  metadata: {}
  outputContract: null
  outputState: null
  outputUtxo: null
  status: confirmed
  tags: {}
  transactionId: 540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd
  txBody: nul

## Transaction 5. The lender withdraws their principal and interest

Here again we withdraw funds from the Marlowe role-payout address.

In [112]:
yaml2json << EOI > request-5.json
contractId: "$CONTRACT_ID"
role: Lender
EOI
cat request-5.json

{"contractId":"97739128440fb360b009d3acbd0eb36a6e5a47da464843413452fd83610282aa#1","role":"Lender"}


Next we post the request and store the response.

In [113]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $LENDER_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

links:
  withdrawal: withdrawals/b09e57eda993ca5a81b1aab3ba5588ea5efaab614a2382f02a7eb170afb872cf
resource:
  txBody:
    cborHex: 86a80083825820540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd02825820540291520e2ad39961a3043e19d40e1b7195809980c0e7c22733e9a84dc494bd03825820dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc020d81825820dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a052a64c0a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a000fc8a0a1581c68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597a1464c656e6465720110a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a367c9d27111a0007ae80021a00051f000b58205507767388bce735e60d8f2c753346ac039cd5d9f029ee3acd35b6b3b81438469fff81d8799f581c68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597464c6

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [114]:
jq '.resource.txBody' response-5.json > tx-5.unsigned

In [115]:
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

TX_5 = b09e57eda993ca5a81b1aab3ba5588ea5efaab614a2382f02a7eb170afb872cf


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [116]:
echo "$EXPLORER_URL/transaction/$TX_5?tab=utxo"

https://preprod.cardanoscan.io/transaction/b09e57eda993ca5a81b1aab3ba5588ea5efaab614a2382f02a7eb170afb872cf?tab=utxo


The lender now has an additional 5 ada (the loan's interest), minus fees, compared to their original balance before the contract started.

In [117]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b09e57eda993ca5a81b1aab3ba5588ea5efaab614a2382f02a7eb170afb872cf     0        86664384 lovelace + TxOutDatumNone
b09e57eda993ca5a81b1aab3ba5588ea5efaab614a2382f02a7eb170afb872cf     1        1034400 lovelace + 1 68d0f8d36ea149b49d28dfa0f631748e41e72d62d46ece4d431ad597.4c656e646572 + TxOutDatumNone
dddc53341eab8f18ff910179912330e95a3a71f5e524e2312e21e38e68d1c9bc     0        914639783 lovelace + TxOutDatumNone


## *Appendix:* OpenApi Description of Marlowe Web Services

Here is the OpenApi description of endpoints and types provided by `marlowe-web-server`.

In [118]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL/openapi.json" | json2yaml

components:
  schemas:
    Action:
      description: A contract which becomes active when an action occurs.
      oneOf:
      - properties:
          deposits:
            $ref: '#/components/schemas/Value'
          into_account:
            $ref: '#/components/schemas/Party'
          of_token:
            $ref: '#/components/schemas/Token'
          party:
            $ref: '#/components/schemas/Party'
        required:
        - party
        - deposits
        - of_token
        - into_account
        type: object
      - properties:
          choose_between:
            items:
              $ref: '#/components/schemas/Bound'
            type: array
          for_choice:
            $ref: '#/components/schemas/ChoiceId'
        required:
        - for_choice
        - choose_between
        type: object
      - properties:
          notify_if:
            $ref: '#/components/schemas/Observation'
        required:
        - notify_if
        type: object
    Address:
      descri